In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
import glob

In [2]:
Phil = pd.read_csv('cleaned_Phil.csv', low_memory=False)
Phil

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates
0,365184634,15,2021-01-01 00:05:00,2021-01-01 00:20:00,3125,3157,14542,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.943909,-75.167351","39.925449,-75.159538"
1,365184633,14,2021-01-01 00:06:00,2021-01-01 00:20:00,3022,3010,5294,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.95472,-75.183228","39.947109,-75.166183"
2,365184632,33,2021-01-01 00:07:00,2021-01-01 00:40:00,3066,3201,14586,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.94561,-75.173477","39.955231,-75.166199"
3,365184631,9,2021-01-01 00:13:00,2021-01-01 00:22:00,3045,3034,14683,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947922,-75.162369","39.933151,-75.162483"
4,365184630,13,2021-01-01 00:37:00,2021-01-01 00:50:00,3052,3040,3330,30,One Way,Indego30,standard,2021-01-01,4,0,2021-01-01,4,0,"39.947319,-75.156952","39.962891,-75.166061"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678837,579499101,23,2022-12-31 23:44:00,2023-01-01 00:07:00,3244,3287,14674,30,One Way,Indego30,standard,2022-12-31,5,23,2023-01-01,6,0,"39.938648,-75.16674","39.943668,-75.216057"
1678838,579499099,17,2022-12-31 23:45:00,2023-01-01 00:02:00,3295,3054,23113,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950279,-75.160271","39.962502,-75.174202"
1678839,579499097,8,2022-12-31 23:51:00,2022-12-31 23:59:00,3255,3046,16897,30,One Way,Indego30,electric,2022-12-31,5,23,2022-12-31,5,23,"39.950951,-75.164383","39.950119,-75.144722"
1678840,579499095,23,2022-12-31 23:54:00,2023-01-01 00:16:00,3255,3113,22646,30,One Way,Indego30,electric,2022-12-31,5,23,2023-01-01,6,0,"39.950951,-75.164383","39.97472,-75.197807"


In [3]:
Phil['start_time'] = pd.to_datetime(Phil['start_time'])
Phil ['end_time'] = pd.to_datetime(Phil['end_time'])

In [4]:
Phil = Phil.sort_values(['bike_id', 'start_time'])
Phil = Phil.reset_index(drop=True)
Phil['idle_time'] = np.nan

**Idle time calculation for a bike after the trip**

In [5]:
trip_ids = set()
lastBikeID = 1678841

for i, row in Phil.iterrows():
    currentBikeID = row['bike_id']
    
    if lastBikeID == i or currentBikeID != Phil.at[i+1, 'bike_id']:
        trip_ids.add(row['trip_id'])
    else:
        Phil.at[i, 'idle_time'] = Phil.at[i+1, 'start_time'] - row['end_time']

In [6]:
Phil

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,idle_time
0,365315729,5,2021-01-03 15:03:00,2021-01-03 15:08:00,3211,3054,102,30,One Way,Indego30,standard,2021-01-03,6,15,2021-01-03,6,15,"39.971722,-75.170601","39.962502,-75.174202",0 days 07:23:00
1,365351230,4,2021-01-03 22:31:00,2021-01-03 22:35:00,3054,3014,102,365,One Way,Indego365,standard,2021-01-03,6,22,2021-01-03,6,22,"39.962502,-75.174202","39.958858,-75.173691",0 days 19:25:00
2,365375491,7,2021-01-04 18:00:00,2021-01-04 18:07:00,3014,3163,102,30,One Way,Indego30,standard,2021-01-04,0,18,2021-01-04,0,18,"39.958858,-75.173691","39.949741,-75.180969",25 days 21:11:00
3,366822049,27,2021-01-30 15:18:00,2021-01-30 15:45:00,3004,3057,102,1,One Way,Day Pass,standard,2021-01-30,5,15,2021-01-30,5,15,"39.953781,-75.163742","39.96439,-75.179871",5 days 17:37:00
4,367159233,17,2021-02-05 09:22:00,2021-02-05 09:39:00,3057,3111,102,365,One Way,Indego365,standard,2021-02-05,4,9,2021-02-05,4,9,"39.96439,-75.179871","39.977791,-75.213226",20 days 06:17:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678837,519832246,105,2022-09-07 09:37:00,2022-09-07 11:22:00,3155,3168,WAND_DENISE,365,One Way,Indego365,standard,2022-09-07,2,9,2022-09-07,2,11,"39.940182,-75.154419","39.95134,-75.173943",NaN
1678838,368796659,128,2021-03-02 14:16:00,2021-03-02 16:24:00,3209,3010,WAND_ERIN_47,30,One Way,Indego30,standard,2021-03-02,1,14,2021-03-02,1,16,"39.949001,-75.212784","39.947109,-75.166183",NaN
1678839,391010723,48,2021-06-06 14:53:00,2021-06-06 15:41:00,3206,3211,WAND_LILRUSS_43,1,One Way,Day Pass,standard,2021-06-06,6,14,2021-06-06,6,15,"39.95174,-75.170631","39.971722,-75.170601",NaN
1678840,407706851,6,2021-07-31 16:14:00,2021-07-31 16:20:00,3108,3101,WAND_PHIL_61,1,One Way,Day Pass,standard,2021-07-31,5,16,2021-07-31,5,16,"39.953159,-75.165512","39.942951,-75.159554",NaN


In [8]:
Phil['idle_time'] = pd.to_timedelta(Phil['idle_time'])
mean_idle_times = Phil.groupby('bike_id')['idle_time'].mean().dt.round('S')

In [9]:
for trip_id in trip_ids:
    last_trip = Phil['trip_id'] == trip_id
    bike_id = Phil.loc[last_trip, 'bike_id'].iloc[0]
    if bike_id in mean_idle_times.index:
        mean_idle_time = mean_idle_times[bike_id]
        Phil.loc[last_trip, 'idle_time'] = mean_idle_time

In [10]:
len(trip_ids)

2373

In [11]:
mask = (Phil['trip_id'].isin(trip_ids))
filtered_data = Phil[mask]
filtered_data

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,idle_time
71,392166834,3,2021-06-10 14:36:00,2021-06-10 14:39:00,3025,3030,102,30,One Way,Indego30,standard,2021-06-10,3,14,2021-06-10,3,14,"39.937241,-75.161201","39.93935,-75.157158",2 days 05:00:13
1196,579447359,8,2022-12-31 18:29:00,2022-12-31 18:36:00,3101,3010,11004,365,One Way,Indego365,standard,2022-12-31,5,18,2022-12-31,5,18,"39.942951,-75.159554","39.947109,-75.166183",0 days 14:59:17
1370,388549130,47,2021-05-28 12:07:00,2021-05-28 12:54:00,3021,3206,11005,30,One Way,Indego30,standard,2021-05-28,4,12,2021-05-28,4,12,"39.953899,-75.169022","39.95174,-75.170631",0 days 19:45:52
2588,575420118,12,2022-12-15 17:38:00,2022-12-15 17:50:00,3158,3237,11006,30,One Way,Indego30,standard,2022-12-15,3,17,2022-12-15,3,17,"39.925522,-75.169037","39.917171,-75.170959",0 days 13:45:17
3542,498225462,20,2022-07-16 11:01:00,2022-07-16 11:21:00,3043,3261,11007,30,One Way,Indego30,standard,2022-07-16,5,11,2022-07-16,5,11,"39.93082,-75.174744","39.963039,-75.140991",0 days 13:45:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678837,519832246,105,2022-09-07 09:37:00,2022-09-07 11:22:00,3155,3168,WAND_DENISE,365,One Way,Indego365,standard,2022-09-07,2,9,2022-09-07,2,11,"39.940182,-75.154419","39.95134,-75.173943",NaT
1678838,368796659,128,2021-03-02 14:16:00,2021-03-02 16:24:00,3209,3010,WAND_ERIN_47,30,One Way,Indego30,standard,2021-03-02,1,14,2021-03-02,1,16,"39.949001,-75.212784","39.947109,-75.166183",NaT
1678839,391010723,48,2021-06-06 14:53:00,2021-06-06 15:41:00,3206,3211,WAND_LILRUSS_43,1,One Way,Day Pass,standard,2021-06-06,6,14,2021-06-06,6,15,"39.95174,-75.170631","39.971722,-75.170601",NaT
1678840,407706851,6,2021-07-31 16:14:00,2021-07-31 16:20:00,3108,3101,WAND_PHIL_61,1,One Way,Day Pass,standard,2021-07-31,5,16,2021-07-31,5,16,"39.953159,-75.165512","39.942951,-75.159554",NaT


In [14]:
average_idle_time = Phil.groupby('bike_id')['idle_time'].mean().dt.round('S')
print(average_idle_time.to_string(float_format='%.2f'))

bike_id
102                2 days 05:00:13
11004              0 days 14:59:17
11005              0 days 19:45:52
11006              0 days 13:45:17
11007              0 days 13:45:10
11008              0 days 20:42:52
11009              0 days 13:23:52
11010              0 days 11:58:05
11011              0 days 11:10:30
11012              0 days 12:06:15
11013              0 days 15:57:35
11014              0 days 13:49:27
11015              0 days 12:19:08
11016              0 days 11:07:01
11017              0 days 12:47:22
11018              0 days 16:04:28
11019              0 days 14:32:03
11020              0 days 11:36:58
11023              0 days 10:19:56
11024              0 days 16:17:09
11025              0 days 16:49:47
11027              0 days 12:05:07
11028              0 days 14:55:14
11029              0 days 12:36:36
11030              0 days 12:36:47
11031              0 days 14:26:27
11033              0 days 12:00:54
11036              0 days 12:18:14
11037       

In [15]:
mask = (Phil['trip_id'].isin(trip_ids)) & (Phil['idle_time'].isna())
filtered_data = Phil[mask]
filtered_data

,trip_id,duration,start_time,end_time,start_station,end_station,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Date_s,Weekday_s,Hour_s,Date_e,Weekday_e,Hour_e,start_coordinates,end_coordinates,idle_time
921145,365200132,11,2021-01-01 12:39:00,2021-01-01 12:50:00,3040,3041,19621,30,One Way,Indego30,electric,2021-01-01,4,12,2021-01-01,4,12,"39.962891,-75.166061","39.968491,-75.13546",NaT
1049048,494116817,43,2022-06-29 23:49:00,2022-06-30 00:32:00,3187,3097,22275,365,One Way,Indego365,electric,2022-06-29,2,23,2022-06-30,3,0,"39.957249,-75.172318","39.978882,-75.133392",NaT
1270238,365260787,55,2021-01-02 14:43:00,2021-01-02 15:38:00,3102,3049,3309,1,One Way,Day Pass,standard,2021-01-02,5,14,2021-01-02,5,15,"39.96759,-75.17952","39.945091,-75.142502",NaT
1678835,431246754,158,2021-10-08 08:36:00,2021-10-08 11:14:00,3203,3160,WAND_CARA_45,30,One Way,Indego30,standard,2021-10-08,4,8,2021-10-08,4,11,"39.940769,-75.172272","39.956619,-75.198624",NaT
1678836,556807927,5,2022-11-04 13:33:00,2022-11-04 13:39:00,3158,3158,WAND_CARNIESHA,365,Round Trip,Indego365,standard,2022-11-04,4,13,2022-11-04,4,13,"39.925522,-75.169037","39.925522,-75.169037",NaT
1678837,519832246,105,2022-09-07 09:37:00,2022-09-07 11:22:00,3155,3168,WAND_DENISE,365,One Way,Indego365,standard,2022-09-07,2,9,2022-09-07,2,11,"39.940182,-75.154419","39.95134,-75.173943",NaT
1678838,368796659,128,2021-03-02 14:16:00,2021-03-02 16:24:00,3209,3010,WAND_ERIN_47,30,One Way,Indego30,standard,2021-03-02,1,14,2021-03-02,1,16,"39.949001,-75.212784","39.947109,-75.166183",NaT
1678839,391010723,48,2021-06-06 14:53:00,2021-06-06 15:41:00,3206,3211,WAND_LILRUSS_43,1,One Way,Day Pass,standard,2021-06-06,6,14,2021-06-06,6,15,"39.95174,-75.170631","39.971722,-75.170601",NaT
1678840,407706851,6,2021-07-31 16:14:00,2021-07-31 16:20:00,3108,3101,WAND_PHIL_61,1,One Way,Day Pass,standard,2021-07-31,5,16,2021-07-31,5,16,"39.953159,-75.165512","39.942951,-75.159554",NaT
1678841,433129426,52,2021-10-13 09:52:00,2021-10-13 10:44:00,3074,3032,WAND_RUSSEL_46,30,One Way,Indego30,standard,2021-10-13,2,9,2021-10-13,2,10,"39.955109,-75.209869","39.945271,-75.17971",NaT


In [16]:
Phil.to_csv('Phil_with_idle_time_bike.csv', index=False)